In [34]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

message = "hi, how are you?\n"

# system_message = '''You are hired as a data scientist on a new VALORANT esports team and 
#     have been tasked by the team’s general manager to support the scouting and recruitment 
#     process, please answer the above question.
#     '''
# message += system_message

body = json.dumps({
    "inputText": message,
    "textGenerationConfig": {
        "maxTokenCount": 2048,
        "stopSequences": ["User:"],
        "temperature": 0,
        "topP": 0.9
    }
})

modelId = 'amazon.titan-text-express-v1'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

output_text = response_body['results'][0]['outputText']

output_text

'Hello! How can I assist you today?'

In [23]:
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 7,
 'results': [{'tokenCount': 9,
   'outputText': 'Hello! How can I help you?',
   'completionReason': 'FINISH'}]}

In [26]:
# 从response_body中获取outputText
output_text = response_body['results'][0]['outputText']
output_text

'Hello! How can I help you?'

In [38]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

message = "hi, how are you?\n"

system_message = '''You are hired as a data scientist on a new VALORANT esports team and 
    have been tasked by the team’s general manager to support the scouting and recruitment 
    process, please answer the above question.
    '''

message += system_message

body = json.dumps({
    "inputText": message,
    "textGenerationConfig": {
        "maxTokenCount": 2048,
        "stopSequences": ["User:"],
        "temperature": 0,
        "topP": 0.9
    }
})

modelId = 'amazon.titan-text-express-v1'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)

stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        print(chunk)
        if chunk:
            print(json.loads(chunk.get('bytes')).get('outputText'), end='')


The VALORANT esports team is looking to hire a data scientist to support the scouting and recruitment process. The general manager has tasked the data scientist with providing insights and analysis to help the team identify and recruit talented players. The data scientist will work closely with the team’s coaches and management to gather and analyze player data, including game statistics, performance metrics, and video footage. They will use advanced analytics techniques to identify patterns and trends in player performance and develop predictive models to help the team make informed decisions about player recruitment. The data scientist will also be responsible for developing and maintaining a database of player information and statistics, which will be used by the team’s coaching staff and management to track player progress and make adjustments to the team’s strategy. In addition to their technical skills, the data scientist will need to have strong communication and interpersonal 

In [34]:
import logging
import boto3

from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_conversation(bedrock_client,
                          model_id,
                          system_prompts,
                          messages):
    """
    Sends messages to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_prompts (JSON) : The system prompts for the model to use.
        messages (JSON) : The messages to send to the model.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logger.info("Generating message with model %s", model_id)

    # Inference parameters to use.
    temperature = 0.5
    top_k = 0.9

    # Base inference parameters to use.
    inference_config = {"temperature": temperature}
    # Additional inference parameters to use.
    additional_model_fields = {"top_k": top_k}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config
        # additionalModelRequestFields=additional_model_fields
    )

    # Log token usage.
    token_usage = response['usage']
    logger.info("Input tokens: %s", token_usage['inputTokens'])
    logger.info("Output tokens: %s", token_usage['outputTokens'])
    logger.info("Total tokens: %s", token_usage['totalTokens'])
    logger.info("Stop reason: %s", response['stopReason'])

    return response

def main():
    model_id = 'meta.llama3-70b-instruct-v1:0'

    # Setup the system prompts and messages to send to the model.
    system_prompts = [{"text": "You are an app that creates playlists for a radio station that plays rock and pop music."
                       "Only return song names and the artist."}]
    message_1 = {
        "role": "user",
        "content": [{"text": "Create a list of 3 pop songs."}]
    }
    message_2 = {
        "role": "user",
        "content": [{"text": "Make sure the songs are by artists from the United Kingdom."}]
    }
    messages = []

    try:

        bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")

        # Start the conversation with the 1st message.
        messages.append(message_1)
        response = generate_conversation(
            bedrock_client, model_id, system_prompts, messages)

        # Add the response message to the conversation.
        output_message = response['output']['message']
        messages.append(output_message)

        # Continue the conversation with the 2nd message.
        messages.append(message_2)
        response = generate_conversation(
            bedrock_client, model_id, system_prompts, messages)

        output_message = response['output']['message']
        messages.append(output_message)

        # Show the complete conversation.
        for message in messages:
            print(f"Role: {message['role']}")
            for content in message['content']:
                print(f"Text: {content['text']}")
            print()

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(
            f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()

INFO:__main__:Generating message with model meta.llama3-70b-instruct-v1:0
INFO:__main__:Input tokens: 49
INFO:__main__:Output tokens: 49
INFO:__main__:Total tokens: 98
INFO:__main__:Stop reason: end_turn
INFO:__main__:Generating message with model meta.llama3-70b-instruct-v1:0
INFO:__main__:Input tokens: 118
INFO:__main__:Output tokens: 51
INFO:__main__:Total tokens: 169
INFO:__main__:Stop reason: end_turn


Role: user
Text: Create a list of 3 pop songs.

Role: assistant
Text: 

Here are 3 pop songs:

1. "Happy" by Pharrell Williams
2. "Uptown Funk" by Mark Ronson ft. Bruno Mars
3. "Can't Stop the Feeling!" by Justin Timberlake

Role: user
Text: Make sure the songs are by artists from the United Kingdom.

Role: assistant
Text: 

Here are 3 pop songs by artists from the United Kingdom:

1. "Wannabe" by Spice Girls
2. "Back for Good" by Take That
3. "Price Tag" by Jessie J ft. B.o.B

Finished generating text with model meta.llama3-70b-instruct-v1:0.


In [42]:
def generate_conversation(bedrock_client,
                          model_id,
                          system_prompts,
                          messages):
    """
    Sends messages to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_prompts (JSON) : The system prompts for the model to use.
        messages (JSON) : The messages to send to the model.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logger.info("Generating message with model %s", model_id)

    # Inference parameters to use.
    temperature = 0.5
    top_k = 0.9

    # Base inference parameters to use.
    inference_config = {"temperature": temperature}
    # Additional inference parameters to use.
    additional_model_fields = {"top_k": top_k}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config
        # additionalModelRequestFields=additional_model_fields
    )

    # Log token usage.
    token_usage = response['usage']
    logger.info("Input tokens: %s", token_usage['inputTokens'])
    logger.info("Output tokens: %s", token_usage['outputTokens'])
    logger.info("Total tokens: %s", token_usage['totalTokens'])
    logger.info("Stop reason: %s", response['stopReason'])

    return response

model_id = "meta.llama3-70b-instruct-v1:0"
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")

system_message = '''
You are an app that creates playlists for a radio station that plays rock and pop music.
Only return song names and the artist.
'''
system_message = [{"text": system_message}]

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 pop songs."}]
}

messages = []
messages.append(message_1)

response = generate_conversation(
    bedrock_client, model_id, system_message, messages)

response


INFO:__main__:Generating message with model meta.llama3-70b-instruct-v1:0
INFO:__main__:Input tokens: 49
INFO:__main__:Output tokens: 53
INFO:__main__:Total tokens: 102
INFO:__main__:Stop reason: end_turn


{'ResponseMetadata': {'RequestId': '5ccf93fc-814f-4d5c-97ce-5f0fe152adc2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 22 Oct 2024 12:53:40 GMT',
   'content-type': 'application/json',
   'content-length': '371',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5ccf93fc-814f-4d5c-97ce-5f0fe152adc2'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': '\n\nHere are 3 pop songs for your radio station:\n\n1. "Happy" by Pharrell Williams\n2. "Uptown Funk" by Mark Ronson ft. Bruno Mars\n3. "Can\'t Stop the Feeling!" by Justin Timberlake'}]}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 49, 'outputTokens': 53, 'totalTokens': 102},
 'metrics': {'latencyMs': 1196}}

In [49]:
response['output']['message']['content'][0]['text']

'\n\nHere are 3 pop songs for your radio station:\n\n1. "Happy" by Pharrell Williams\n2. "Uptown Funk" by Mark Ronson ft. Bruno Mars\n3. "Can\'t Stop the Feeling!" by Justin Timberlake'